Inside `ItemList._label_from_list`, the list of subfoldernames(strings) becomes an array of labels(strings, such as '7', '9'), and then uses `ItemList.get_label_cls(labels)` to figure out the label class is `CategoryList`. Then it uses the array of label strings and `CategoryList` to initialize an `CategoryList` object named `y` for all the labels. It's important to note that `ItemList._label_from_list` use input argument `from_item_lists` to help ensure we use `ItemLists` with `label_from_folder`. 

and the `CategoryList` object or `y` is actually an `ItemList` object (subclass of `CategoryListBase` which is a subclass of `ItemList`) with some additional properties, such as `classes` (default as `None`), `filter_missing_y` (default as `True`), `loss_func` (default as `CrossEntropyFlat()`), and with class attribute `_processor` (default as `CategoryProcessor`), meanwhile the array of label strings become this `ItemList.items`.

The final step of `ItemList._label_from_list` is to initialize a `LabelList` object with `LabelList(x, y, tfms=None, tfm_y=False)`. 

`LabelList(x, y, tfms=None, tfm_y=False)` basically creates a few properties, `LabelList.x`, `LabelList.y`, `LabelList.tfm_y`, `LabelList.y.x`, `LabelList.item`, and then use `LabelList.transform()` only to set values for `LabelList.tfm_y`, `LabelList.tfms`, `LabelList.tfmargs`, `LabelList.tfmargs_y`.

So finally, `ItemList.label_from_folder()` turns `ItemList` into a `LabelList` which has a property `x` (a `ImageList`) and y (a `CategoryList`) in the example above.

In [28]:
ll_train = il_train.label_from_folder(from_item_lists=True)

We can force to create a `LabelList` as above, but many values are missing, such as  `ll_train.y.classes` is `None`, therefore `ll_train.y.c` is not available, and `ll_train.y.items` is an array of strings not numbers (which it should be). As a result, such a `LabelList` won't be printed out properly.

So when doing it properly using `ItemLists.label_from_folder()`, inside `_inner()`, `ItemLists` is turned into a `LabelLists` and calls `LabelLists.process()` to process the inner dataset.

Insdie `LabelLists.process()`, it first gets processors for both x and y, in this case `xp` is `None`, `yp` is `CategoryProcessor` which got initialized as well. Secondly, loop through the training, validation and testing (optionally) `LabelList` objects and apply x and y processors to their x and y datasets with `LabelList.process`.

Inside `LabelList.process`, the first task is for `CategoryProcessor.process` to process `y`, or the `CategoryList`. In the process, it first gets the unique strings out of `y.items` (array of strings for labels), and then `CategoryList` takes the unqiue strings as its `classes` properties and created property `c2i` (class to index) with a dictionary. Thirdly, it passes these two properties above to be `y` properties. Lastly, `PreProcessor.process` converts `y.items` from an array of label/class strings to an array of class index. After this processing, `CategoryList` properties such as `c`, `classes`, and `items` are properly updated, and `y` can get properly printed out. 

the second task of `LabelList.process`, if there are some missing values in `CategoryList`, `y`, it will filter out missing values for both x and y. The final task, is to use `xp` to process `x`. 

In [42]:
ll_train.y._processor

fastai.data_block.CategoryProcessor

In [39]:
ll.train.y.items

array([1, 1, 1, 1, ..., 0, 0, 0, 0])